### Applications of Spectral Graphs Theory: Google Page Ranking
---
Read the hand-written notes on Page ranking on Module 2 content. 

Reference 1: Numerical Linear Algebra in Data Mining-[Elden](http://www.cato.tzo.com/brad_bbk/teaching/datamining/research_surveys/elden.pdf), Page 40-46.

Reference 2: Please see [this link](https://rstudio-pubs-static.s3.amazonaws.com/239261_8a607707294341c4b7e26acf728c28bd.html) on power iteration and page-rank.

---

In [20]:
import numpy as np
from scipy.sparse import coo_matrix
from scipy.sparse import csr_matrix

np.set_printoptions(precision=5)

**Power-iteration** [C-Engage Chapter](https://college.cengage.com/mathematics/larson/elementary_linear/5e/students/ch08-10/chap_10_3.pdf)

An iterative algorithm to find the dominant eigenvalue and associated eigenvector of certain matrices.

Start with a random initialization $v_0$ repeat until convergence

$$\large
v_{k+1} = \frac{Av_k}{\|Av_k\|} ;\qquad \lambda_{k} = \frac{v_{k+1}Av_{k}}{v_{k+1}v_{k}}
$$

Necessary and sufficient condition for convergence
>1. The dominant eigenvalue must  be larger than every other eigenvalue in magnitude. rate of convergence is proportional to $|\lambda_2/\lambda_1|$
>2. The initialization $v_0$ must have a nonzero component along the eigenvector associated with the dominant eigenvalue.

In [27]:
# Implementation of Vanilla power method
def power_method(A, max_iter):
    # Input A: the stochastic matrix after adjusting for teleportation
    v_old = np.random.rand(A.shape[1]) # Why should you choose randomly
    #csrA = csr_matrix(A)
    for k in range(max_iter):
        v_new = A.dot(v_old)

        # Need to normalize
        v_new = v_new / np.linalg.norm(v_new)
        # Once can break the loop based on some tolerance criteria. Otherwsie
        v_old = v_new
        
    evalue = np.dot(v_new,A.dot(v_new))/np.linalg.norm(v_new)
    return v_new, evalue

In [51]:
## Implementation of power method customized for page ranking. See Elden.
def power_method_ranking(Q, max_iter):
    # Input Q: The link matrix without any adjustments
    n = Q.shape[1]
    Q_csr = csr_matrix(Q)
    z = np.random.rand(n) # Why should you choose randomly
    z = z / np.linalg.norm(z,ord=1) # z should be of unit 1-norm.
    #print(z.shape)
    alpha=0.85 # Damping factor
    # Peronalization vector must be of unit norm
    v = np.ones((Q.shape[1],), dtype=float)
    v = v / np.linalg.norm(v,ord=1) 

    # Power iteration loop
    for k in range(max_iter):
        y=alpha*Q_csr.dot(z)
        #print(y.shape)
        beta = 1.0 - np.linalg.norm(y,ord=1)
        #print(beta.shape)
        z = y + beta * v
        #print(z.shape)
        residual = np.linalg.norm(y-z,ord=1)
        if residual < 0.0001:
            break
    return z

**Example** Consider the following toy worldwide web to answer the following.
<img src="./images/PageRankingProb1.png" width="60%"/>
- Create the link matrix $Q$ from the toy worldwide web.
- Find the modified link matrix $P$ and page-rank $A$ by assuming a damping factor of $\alpha=0.85$. 
- Find the dominant eigenvector of $A$.
- Arrange the webpages in the decreasing order of their page rank.

<strong>Link Matrix</strong>
<div style="display:" >
    $$
Q = 
\begin{pmatrix}
0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\
1/2 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\
1/2 & 0 & 0 & 0 & 1/2 & 0 & 1/3 & 0 \\
0 & 0 & 0 & 0 & 0 & 1/3 & 0 & 1 \\
0 & 0 & 0 & 0 & 0 & 1/3 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 1/3 & 0 \\
0 & 0 & 0 & 0 & 1/2 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 1/3 & 1/3 & 0 \\
\end{pmatrix}
$$
</div>


<strong>Transition Probability Matrix:</strong> What is wrong in the following?
$$
P = 
\begin{pmatrix}
0 & 1 & 0 & 1/8 & 0 & 0 & 0 & 0 \\
1/2 & 0 & 1 & 1/8 & 0 & 0 & 0 & 0 \\
1/2 & 0 & 0 & 1/8 & 1/2 & 0 & 1/3 & 0 \\
0 & 0 & 0 & 1/8 & 0 & 1/3 & 0 & 1 \\
0 & 0 & 0 & 1/8 & 0 & 1/3 & 0 & 0 \\
0 & 0 & 0 & 1/8 & 0 & 0 & 1/3 & 0 \\
0 & 0 & 0 & 1/8 & 1/2 & 0 & 0 & 0 \\
0 & 0 & 0 & 1/8 & 0 & 1/3 & 1/3 & 0 \\
\end{pmatrix}
$$

<div class="alert alert-success">
<strong>Page Ranking Matrix</strong>
$$
A = \alpha P + \frac{1-\alpha}{n} e e^T =  0.85 P + \frac{0.15}{n} e e^T
$$
    </div>

In [52]:
## Number of nodes (pages)
n = 8
# Please note that one could directly create the 8x8 square matrix P from the graph

# Create the relevant matrices for eigenvector calculation and page ranking
row= [0,0,1,1,1,2,2,2,2,3,3,3,4,4,5,5,6,6,7,7,7]
col = [1,3,0,2,3,0,3,4,6,3,5,7,3,5,3,6,3,4,3,5,6]
val = [1,1/8,1/2,1,1/8,1/2,1/8,1/2,1/3,1/8,1/3,1,1/8,1/3,1/8,1/3,1/8,1/2,1/8,1/3,1/3]

P_coo = coo_matrix((val,(row,col)),(n,n), dtype=float)
P = P_coo.toarray()
print("The modified link matrix:\n",P)
P_csr = P_coo.tocsc()

The modified link matrix:
 [[0.      1.      0.      0.125   0.      0.      0.      0.     ]
 [0.5     0.      1.      0.125   0.      0.      0.      0.     ]
 [0.5     0.      0.      0.125   0.5     0.      0.33333 0.     ]
 [0.      0.      0.      0.125   0.      0.33333 0.      1.     ]
 [0.      0.      0.      0.125   0.      0.33333 0.      0.     ]
 [0.      0.      0.      0.125   0.      0.      0.33333 0.     ]
 [0.      0.      0.      0.125   0.5     0.      0.      0.     ]
 [0.      0.      0.      0.125   0.      0.33333 0.33333 0.     ]]


In [54]:
# The page ranking matrix
alpha=0.85
A = alpha*P + (1.0-alpha)*np.ones_like(P)/P.shape[0]

print(A)

[[0.01875 0.86875 0.01875 0.125   0.01875 0.01875 0.01875 0.01875]
 [0.44375 0.01875 0.86875 0.125   0.01875 0.01875 0.01875 0.01875]
 [0.44375 0.01875 0.01875 0.125   0.44375 0.01875 0.30208 0.01875]
 [0.01875 0.01875 0.01875 0.125   0.01875 0.30208 0.01875 0.86875]
 [0.01875 0.01875 0.01875 0.125   0.01875 0.30208 0.01875 0.01875]
 [0.01875 0.01875 0.01875 0.125   0.01875 0.01875 0.30208 0.01875]
 [0.01875 0.01875 0.01875 0.125   0.44375 0.01875 0.01875 0.01875]
 [0.01875 0.01875 0.01875 0.125   0.01875 0.30208 0.30208 0.01875]]


In [55]:
# Ranking by using Numpy for eigenvector calculation
e,v = np.linalg.eig(A)
print("\n The eigenvalues are:\n",e)
#print("The dominant eigenvector:\n", v[:,0])
print("\n The dominant eigenvector after normalization:\n", np.real(v[:,0])/np.linalg.norm(v[:,0],ord=1))
print("\n Raking from lower to higher:\n",np.argsort(np.abs(v[:,0]))+1)


 The eigenvalues are:
 [ 1.     +0.j       0.62835+0.j      -0.425  +0.425j   -0.425  -0.425j
 -0.11835+0.24013j -0.11835-0.24013j -0.1427 +0.21923j -0.1427 -0.21923j]

 The dominant eigenvector after normalization:
 [-0.27649 -0.29291 -0.174   -0.08245 -0.03884 -0.03998 -0.04402 -0.05131]

 Raking from lower to higher:
 [5 6 7 8 4 3 1 2]


In [56]:
# Vanila power method defined below
z,_ = power_method(A,1000)
#print(z.shape)
z = z / np.linalg.norm(z,ord=1)
print("The dominant eigenvector:\n", z)
print("\n Raking from lower to higher:",np.argsort(z)+1)

The dominant eigenvector:
 [0.27649 0.29291 0.174   0.08245 0.03884 0.03998 0.04402 0.05131]

 Raking from lower to higher: [5 6 7 8 4 3 1 2]


In [57]:
# By using optimized power method. You need to pass the link matrix Q.
Q = np.copy(P)
Q[:,3] = np.zeros(Q.shape[0],dtype=float)
z = power_method_ranking(Q,1000)

print("The dominant eigenvector:\n", z)
print("Raking from lower to higher:",np.argsort(z)+1)

## Hello everyone, please check that both power methods provide the same dominant eigenvectors now.

The dominant eigenvector:
 [0.27649 0.29291 0.174   0.08245 0.03884 0.03998 0.04402 0.05131]
Raking from lower to higher: [5 6 7 8 4 3 1 2]


## Second Example
---

In [46]:

Q = np.array([[0, 1/4, 0, 0, 0, 0, 0, 0],
              [1/4, 0, 1/2, 0, 0, 0, 1, 0],
              [0, 0, 0, 0, 1/2, 0, 0, 1/2],
              [1/4, 0, 0, 0, 0, 0, 0, 0],
              [0, 1/4, 0, 1/3, 0, 1/3, 0, 0],
              [1/4, 1/4, 1/2, 0, 0, 0, 0, 1/2],
              [1/4, 0, 0, 1/3, 1/2, 1/3, 0, 0],
              [0, 1/4, 0, 1/3, 0, 1/3, 0, 0]],dtype=float)
print ("The link matrix \n",Q)

The link matrix 
 [[0.      0.25    0.      0.      0.      0.      0.      0.     ]
 [0.25    0.      0.5     0.      0.      0.      1.      0.     ]
 [0.      0.      0.      0.      0.5     0.      0.      0.5    ]
 [0.25    0.      0.      0.      0.      0.      0.      0.     ]
 [0.      0.25    0.      0.33333 0.      0.33333 0.      0.     ]
 [0.25    0.25    0.5     0.      0.      0.      0.      0.5    ]
 [0.25    0.      0.      0.33333 0.5     0.33333 0.      0.     ]
 [0.      0.25    0.      0.33333 0.      0.33333 0.      0.     ]]


In [47]:
#Example
P = np.copy(Q)
n=P.shape[0]
alpha = 0.85
A = alpha * P + np.ones_like(P, dtype=float)*(1-alpha)/n
#print(P)
#print(A)
lmbda,V = np.linalg.eig(P)
print("Check v and Pv must be the same\n ",V[:,0])
print(P@V[:,0])

Check v and Pv must be the same
  [0.13949+0.j 0.55798+0.j 0.31386+0.j 0.03487+0.j 0.31386+0.j 0.48823+0.j
 0.36617+0.j 0.31386+0.j]
[0.13949+0.j 0.55798+0.j 0.31386+0.j 0.03487+0.j 0.31386+0.j 0.48823+0.j
 0.36617+0.j 0.31386+0.j]


In [48]:
#v,eig_1 = power_method(P,1000)
#print(v)
#print(P@v)
z= power_method_ranking(Q,1000)
print("The dominant eigenvector:\n",z)
print("Raking from lower to higher:",np.argsort(z)+1)

The dominant eigenvector:
 [0.06295 0.20799 0.12367 0.03213 0.12343 0.18134 0.14507 0.12343]
Raking from lower to higher: [4 1 5 8 3 7 6 2]


In [50]:
z,_ = power_method(A,1000)
#print(z.shape)
z = z / np.linalg.norm(z,ord=1)
print("The dominant eigenvector:\n", z)
print("Raking from lower to higher:",np.argsort(z)+1)

The dominant eigenvector:
 [0.06295 0.20799 0.12367 0.03213 0.12343 0.18134 0.14507 0.12343]
Raking from lower to higher: [4 1 5 8 3 7 6 2]
